In [1]:
import requests
from requests.exceptions import ConnectionError
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime
from github import Github

In [2]:
# simple function to request HTML text
# require requests & fake_useragent package
def getHTML(url): 
    try:
        ua = UserAgent().random
        header = {'user-agent':ua}
        r = requests.get(url, headers = header, timeout=30)
        r.raise_for_status() # raise error if r.status_code != 200
        r.encoding = r.apparent_encoding
        return r.text
    except ConnectionError:
        pass
    except Exception as e:
        return e

In [3]:
# function to get the NAMEs of the latest N files in Github repo
def updateFILENAMEs(url,matchstr,n):
    try:
#         global fileNAMEs
        html = getHTML(url)
        soup = BeautifulSoup(html,'html.parser')
        # initialise a list to store the filenames
        filenames = []
        for a in soup.find_all('a'):
            text = a.get_text()
            if re.match(matchstr,text) is not None:
                filenames.append(text)
        filenames = filenames[-n:]
        # upload to Github
        # First create a Github instance:
        g = Github("dde59fd520aa7102e518a9b0993e9bca2f8122a2")
        # Creat a repository object:
        repo = g.get_repo("CASA-DV-Group3/AirQuality-0")
        # get the contents of the original file:
        contents = repo.get_contents('assets/data/fileNAMEs.txt',ref='lyu')
        # update the file contents:
        repo.update_file(contents.path,'update latest geojson filenames on '+str(datetime.now()),str(filenames),contents.sha,branch="lyu")
        print(str(datetime.now())+'    '+'success')
        return
    except Exception as e:
        print(e)
        return

In [4]:
# initialise the parameters
url = 'https://github.com/CASA-DV-Group3/AirQuality-0/tree/lyu/assets/data'
matchstr = 'STATIONdata'
n = 12
updateFILENAMEs(url,matchstr,n)
# # set up the scheduler
# scheduler = BackgroundScheduler()
# scheduler.add_job(updateFILENAMEs,'interval', args=[url,matchstr,n], hours=1, start_date='2019-05-20 00:00:00', end_date='2019-05-21 00:00:00')
# scheduler.start()

2019-05-19 22:17:33.687089    success
